In [1]:
!pip install pytesseract


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install spacy


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy
import re
import string

In [4]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation= "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tablewhitespace = str.maketrans('','',whitespace)
    tablepunctuation = str.maketrans('','',punctuation)

    text = str(txt)
    text = text.lower()
    remvoewhitespace = text.translate(tablewhitespace)
    removepunctuation = remvoewhitespace.translate(tablepunctuation)
    return str(removepunctuation)

In [5]:
model_ner = spacy.load('../3. Train Named Entity Recognition/output/model-best')

In [6]:
# load image
image = cv2.imread('6.jpg')

# extract data
testData = pytesseract.image_to_data(image)

#convert data into content
test_list = list(map(lambda x: x.split('\t'), testData.split('\n')))
df = pd.DataFrame(test_list[1:], columns=test_list[0])
df.dropna(inplace=True)
df['text'] = df['text'].apply(cleanText)

#convertdata into content
dataClean = df.query("text != '' ")
content = " ".join([w for w in dataClean['text']])
print(content)

# get predictions from NER model

doc = model_ner(content)

cell 8099948528 te 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrikantht@gmail.com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo


In [7]:
from spacy import displacy

In [8]:
# displacy.serve(doc, style='ent')

In [9]:
displacy.render(doc, style='ent')

In [10]:
# Tagging

docjosn = doc.to_json()
docjosn.keys()

dict_keys(['text', 'ents', 'tokens'])

In [11]:
doc_text = docjosn['text']


In [12]:
dataframe_tokens = pd.DataFrame(docjosn['tokens'])
dataframe_tokens['token'] = dataframe_tokens[['start', 'end']].apply(
    lambda x: doc_text[x[0]:x[1]], axis=1)

In [13]:
dataframe_tokens.head()

,id,start,end,token
0,0,0,4,cell
1,1,5,15,8099948528
2,2,16,18,te
3,3,19,29,8466045457
4,4,30,35,email


In [14]:
right_table = pd.DataFrame(docjosn['ents'])[['start', 'label']]
dataframe_tokens = pd.merge(dataframe_tokens, right_table, how='left', on='start')

In [15]:
dataframe_tokens.fillna('O', inplace=True)
dataframe_tokens.head()

,id,start,end,token,label
0,0,0,4,cell,O
1,1,5,15,8099948528,B-PHONE
2,2,16,18,te,O
3,3,19,29,8466045457,B-PHONE
4,4,30,35,email,O


In [16]:
# join label to dataClean dataframe
dataClean['end'] = dataClean['text'].apply(lambda x : len(x)+1).cumsum() - 1
dataClean['start'] = dataClean[['text', 'end']].apply(lambda x: x[1] - len(x[0]), axis=1)

/tmp/ipykernel_5195/1829804590.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean['end'] = dataClean['text'].apply(lambda x : len(x)+1).cumsum() - 1
/tmp/ipykernel_5195/1829804590.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean['start'] = dataClean[['text', 'end']].apply(lambda x: x[1] - len(x[0]), axis=1)


In [17]:
dataClean.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start
12,5,1,3,1,1,1,722,53,64,28,92,cell,4,0
14,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5
17,5,1,3,2,1,1,55,55,85,89,31,te,18,16
18,5,1,3,2,1,2,822,71,197,67,94,8466045457,29,19
20,5,1,3,2,2,1,593,136,93,25,89,email,35,30


In [18]:
# inner join with start
datarame_info = pd.merge(dataClean, dataframe_tokens[['start', 'token', 'label']], how='inner', on='start')

In [19]:
datarame_info.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
0,5,1,3,1,1,1,722,53,64,28,92,cell,4,0,cell,O
1,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5,8099948528,B-PHONE
2,5,1,3,2,1,1,55,55,85,89,31,te,18,16,te,O
3,5,1,3,2,1,2,822,71,197,67,94,8466045457,29,19,8466045457,B-PHONE
4,5,1,3,2,2,1,593,136,93,25,89,email,35,30,email,O


In [20]:
# Bounding BOx

In [21]:
bb_df = datarame_info.query("label != 'O' ")
bb_df.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label
1,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5,8099948528,B-PHONE
3,5,1,3,2,1,2,822,71,197,67,94,8466045457,29,19,8466045457,B-PHONE
5,5,1,3,2,2,3,709,136,316,31,88,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,B-EMAIL
6,5,1,3,3,1,1,46,170,33,14,96,life,62,58,life,B-ORG
7,5,1,3,3,1,2,85,151,91,42,94,insurance,72,63,insurance,I-ORG


In [22]:
img = image.copy()

In [23]:
for x,y,w,h,label in bb_df[['left', 'top', 'width', 'height', 'label']].values:
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    
    cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
    cv2.putText(img, str(label), (x,y), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 0), 2)
    
cv2.imshow('Predictions', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread

In [24]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])

/tmp/ipykernel_5195/3914511576.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])


In [25]:
# group the label

class groupgen():
    def __init__(self):
        self.id = 0
        self.text = ''
    
    def getgroup(self, text):
        if self.text == text:
            return self.id
        else:
            self.id +=1
            self.text = text
            return self.id

grp_gen = groupgen()

In [26]:
bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)

/tmp/ipykernel_5195/472836701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)


In [27]:
bb_df.head()

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,token,label,group
1,5,1,3,1,1,3,822,53,203,28,96,8099948528,15,5,8099948528,PHONE,1
3,5,1,3,2,1,2,822,71,197,67,94,8466045457,29,19,8466045457,PHONE,1
5,5,1,3,2,2,3,709,136,316,31,88,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,EMAIL,2
6,5,1,3,3,1,1,46,170,33,14,96,life,62,58,life,ORG,3
7,5,1,3,3,1,2,85,151,91,42,94,insurance,72,63,insurance,ORG,3


In [28]:
# right and bottom of bounding box
bb_df[['left', 'top', 'width', 'height']] = bb_df[['left', 'top', 'width', 'height']].astype(int)

bb_df['right'] = bb_df['left'] + bb_df['width']
bb_df['bottom'] = bb_df['top'] + bb_df['height']

/tmp/ipykernel_5195/3828079379.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df[['left', 'top', 'width', 'height']] = bb_df[['left', 'top', 'width', 'height']].astype(int)
/tmp/ipykernel_5195/3828079379.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['right'] = bb_df['left'] + bb_df['width']
/tmp/ipykernel_5195/3828079379.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [29]:
# tagging: groupby group

col_group = ['left', 'top', 'right', 'bottom', 'label', 'token', 'group']
group_tag_img = bb_df[col_group].groupby(by='group')

In [30]:
img_tagging = group_tag_img.agg({
    'left': min,
    'right': max,
    'top': min,
    'bottom': max,
    'label': np.unique,
    'token': lambda x: " ".join(x)
})

In [31]:
img_tagging

,left,right,top,bottom,label,token
group,,,,,,
1,822,1025,53,138,[PHONE],8099948528 8466045457
2,709,1025,136,167,[EMAIL],lictsrikant@gmail.com
3,46,374,151,193,[ORG],life insurance corporation of india
4,668,1025,163,209,[EMAIL],seosrikantht@gmail.com
5,310,755,227,259,[NAME],thathineni srikanth
6,399,669,271,296,[DES],insurance advisor
7,47,882,395,427,[ORG],life insurance corporation of india
8,46,476,506,533,[WEB],lictsrikant8099948528.blogspot.in


In [32]:
img_bb = image.copy()
for l,r,t,b,label,token in img_tagging.values:
    cv2.rectangle(img_bb, (l,t), (r,b), (0,255,0),2)
    cv2.putText(img_bb, str(label), (l,t), cv2.FONT_HERSHEY_PLAIN, 1, (255,0,255), 2)

cv2.imshow("Bounding Box Business Card", img_bb)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread (0x77d5cb0).
Cannot move to target thread (0x6fed7b0)

QObject::moveToThread: Current thread (0x6fed7b0) is not the object's thread

In [33]:
# Parsing

In [38]:
def parser(text, label):
    if label == 'PHONE':
        text = text.lower()
        text = re.sub(r'\D', '', text)
        
    elif label == 'EMAIL':
        text = text.lower()
        allow_special = '@_.\-'
        text = re.sub(r'[^A-Za-z10-9{} ]'.format(allow_special),'',text)
    elif label == 'WEB':
        text = text.lower()
        allow_spe_cha = ':/.%#\-'
        text = re.sub(r'[^A-Za-z10-9{} ]'.format(allow_spe_cha),'',text)
    elif label in ('NAME', 'DES'):
        text = text.lower()
        text = re.sub(r'[^a-z ]','',text)
        text = text.title()
    elif label == 'ORG':
        text = text.lower()
        text = re.sub(r'[^a-z0-9 ]','',text)
        text = text.title()
    
    return text
        

In [39]:
parser('Faruk 3043740', 'NAME')

'Faruk '

In [40]:
# entites

In [41]:
info_array = datarame_info[['token', 'label']].values
entities = dict(NAME=[], ORG=[],DES=[],PHONE=[],EMAIL=[],WEB=[])

previous = 'O'
for token, label in info_array:
    bio_tag = label[:1]
    label_tag = label[2:]
    # step -1 parse the token
    text = parser(token, label_tag)
    
    if bio_tag in ('B', 'I'):
        if previous != label_tag:
            entities[label_tag].append(text)
        else:
            if bio_tag == 'B':
                entities[label_tag].append(text)
            else:
                if label_tag in ("NAME", 'ORG', 'DES'):
                    entities[label_tag][-1] = entities[label_tag][-1] + " " + text
                else:
                    entities[label_tag][-1] = entities[label_tag][-1] + " " + text
    
    previous = label_tag
                

In [42]:
entities

{'NAME': ['Thathineni Srikanth'],
 'ORG': ['Life Insurance Corporation Of India',
  'Life Insurance Corporation Of India'],
 'DES': ['Insurance Advisor'],
 'PHONE': ['8099948528', '8466045457'],
 'EMAIL': ['lictsrikant@gmail.com', 'seosrikantht@gmail.com'],
 'WEB': ['lictsrikant8099948528.blogspot.in']}